# LangGraph Human-in-the-Loop Training for Banking IT Professionals
## Comprehensive Hands-on Lab Exercises

This notebook contains 4 complete lab exercises covering all human-in-the-loop patterns:

1. **Lab 1**: Loan Approval with Static Interrupts (`interrupt_before`)
2. **Lab 2**: Fraud Detection with Dynamic Interrupts (`interrupt()` function)
3. **Lab 3**: Account Opening with State Editing (`update_state()`)
4. **Lab 4**: Customer Support with Multi-turn Conversations

### Prerequisites
```bash
pip install langgraph langchain-groq python-dotenv
```

### Setup
Set your GROQ_API_KEY environment variable before running the labs.

In [ ]:
# Install required packages (run once)
!pip install -q langgraph langchain-groq python-dotenv

In [ ]:
# Setup: Configure your GROQ API Key
import os
from getpass import getpass

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass("Enter your GROQ API Key: ")

print("✅ API Key configured")

---
## Lab 1: Loan Approval with Static Interrupts

### Learning Objectives:
- Understand static interrupts using `interrupt_before` and `interrupt_after`
- Learn checkpoint-based persistence
- Practice reviewing and approving actions before execution

### Banking Scenario:
A loan application system where a human loan officer must review AI-generated risk assessments before final loan decisions are made.

In [ ]:
# Lab 1: Imports
import os
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# Define State
class LoanApplicationState(TypedDict):
    applicant_name: str
    loan_amount: float
    annual_income: float
    credit_score: int
    employment_years: int
    risk_assessment: str
    loan_decision: str
    officer_notes: str
    messages: Annotated[list, "conversation history"]

# Initialize LLM
def get_llm():
    return ChatGroq(
        model="llama-3.1-8b-instant",
        temperature=0.3,
        groq_api_key=os.getenv("GROQ_API_KEY")
    )

print("✅ Lab 1 setup complete")

In [ ]:
# Lab 1: Define Workflow Nodes

def analyze_loan_application(state: LoanApplicationState) -> LoanApplicationState:
    """AI analyzes the loan application and provides risk assessment"""
    llm = get_llm()
    
    system_message = """You are a financial risk analyst. 
    Analyze the loan application and provide a comprehensive risk assessment.
    Consider: credit score, debt-to-income ratio, employment stability.
    
    Risk Categories:
    - LOW RISK: Credit score > 750, 5+ years employment, loan < 30% income
    - MEDIUM RISK: Credit score 650-750, 2-5 years employment, loan 30-50% income  
    - HIGH RISK: Credit score < 650, < 2 years employment, loan > 50% income"""
    
    prompt = f"""
    Loan Application:
    - Applicant: {state['applicant_name']}
    - Loan Amount: ${state['loan_amount']:,.2f}
    - Annual Income: ${state['annual_income']:,.2f}
    - Credit Score: {state['credit_score']}
    - Years of Employment: {state['employment_years']}
    
    Provide your risk assessment and recommendation.
    """
    
    messages = [SystemMessage(content=system_message), HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    print("\n" + "="*80)
    print("🤖 AI RISK ASSESSMENT")
    print("="*80)
    print(response.content)
    print("="*80)
    
    return {
        "risk_assessment": response.content,
        "messages": state.get("messages", []) + [HumanMessage(content=prompt), AIMessage(content=response.content)]
    }

def make_loan_decision(state: LoanApplicationState) -> LoanApplicationState:
    """Make final loan decision based on risk assessment"""
    llm = get_llm()
    
    prompt = f"""
    Make final decision for: {state['applicant_name']}
    Loan Amount: ${state['loan_amount']:,.2f}
    
    Risk Assessment: {state['risk_assessment']}
    Officer Notes: {state.get('officer_notes', 'None')}
    
    Provide: APPROVED or DENIED with terms and reasoning.
    """
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    print("\n" + "="*80)
    print("✅ FINAL LOAN DECISION")
    print("="*80)
    print(response.content)
    print("="*80)
    
    return {"loan_decision": response.content}

print("✅ Nodes defined")

In [ ]:
# Lab 1: Build and Compile Graph

def create_loan_workflow():
    workflow = StateGraph(LoanApplicationState)
    
    # Add nodes
    workflow.add_node("analyze_application", analyze_loan_application)
    workflow.add_node("make_decision", make_loan_decision)
    
    # Define edges
    workflow.add_edge(START, "analyze_application")
    workflow.add_edge("analyze_application", "make_decision")
    workflow.add_edge("make_decision", END)
    
    # Compile with STATIC INTERRUPT
    memory = MemorySaver()
    app = workflow.compile(
        checkpointer=memory,
        interrupt_before=["make_decision"]  # Pause before final decision
    )
    
    return app

print("✅ Workflow created")

In [ ]:
# Lab 1: Run Demo

app = create_loan_workflow()
thread_config = {"configurable": {"thread_id": "loan_001"}}

# Initial application
initial_state = {
    "applicant_name": "Sarah Martinez",
    "loan_amount": 250000.0,
    "annual_income": 85000.0,
    "credit_score": 720,
    "employment_years": 4,
    "risk_assessment": "",
    "loan_decision": "",
    "officer_notes": "",
    "messages": []
}

print("📋 Loan Application Submitted")
print(f"Applicant: {initial_state['applicant_name']}")
print(f"Loan Amount: ${initial_state['loan_amount']:,.2f}")
print(f"Credit Score: {initial_state['credit_score']}\n")

# Run until interrupt
print("🔄 Running workflow...\n")
for event in app.stream(initial_state, thread_config, stream_mode="values"):
    pass

# Check state
current_state = app.get_state(thread_config)
print("\n⏸️  INTERRUPTED - Awaiting Loan Officer Review")
print(f"Next Node: {current_state.next}")
print(f"Risk Assessment Generated: {'Yes' if current_state.values.get('risk_assessment') else 'No'}")

In [ ]:
# Lab 1: Simulate Human Review and Resume

# Loan officer adds notes
officer_notes = """
Approved with conditions:
- Strong income and good credit score
- Solid employment history (4 years)
- Recommend standard rate with 20% down payment
"""

print("👤 LOAN OFFICER REVIEW")
print("="*80)
print(officer_notes)
print("="*80)

# Update state with officer notes
app.update_state(thread_config, {"officer_notes": officer_notes})

# Resume workflow
print("\n▶️  Resuming workflow...\n")
for event in app.stream(None, thread_config, stream_mode="values"):
    pass

print("\n✅ WORKFLOW COMPLETED")

### Lab 1 Exercises:
1. Change interrupt to `interrupt_after=["analyze_application"]`
2. Add a compliance check node with another interrupt
3. Test with high-risk applicant (credit score < 650)
4. Implement rejection handling logic

---
## Lab 2: Fraud Detection with Dynamic Interrupts

### Learning Objectives:
- Use dynamic interrupts with `interrupt()` function
- Implement conditional human-in-the-loop
- Use `Command(resume=...)` to provide input

### Banking Scenario:
Automated fraud detection that only pauses for human review when high fraud risk is detected.

In [ ]:
# Lab 2: Imports and Setup
from langgraph.types import interrupt, Command
import json

class TransactionState(TypedDict):
    transaction_id: str
    account_holder: str
    transaction_amount: float
    merchant: str
    location: str
    transaction_type: str
    account_balance: float
    recent_transactions: list
    fraud_score: float
    fraud_reason: str
    fraud_analyst_decision: str
    final_status: str
    messages: Annotated[list, "conversation history"]

print("✅ Lab 2 setup complete")

In [ ]:
# Lab 2: Define Nodes with Dynamic Interrupt

def analyze_transaction(state: TransactionState) -> TransactionState:
    """Analyze transaction for fraud indicators"""
    llm = get_llm()
    
    system_message = """You are a fraud detection AI.
    Analyze transactions and calculate fraud risk score (0-100).
    High-risk: Large amounts (>$5,000), foreign locations, unusual patterns.
    Return ONLY JSON: {"fraud_score": <number>, "fraud_reason": "<text>"}"""
    
    prompt = f"""
    Transaction: {state['transaction_id']}
    Amount: ${state['transaction_amount']:,.2f}
    Merchant: {state['merchant']}
    Location: {state['location']}
    Balance: ${state['account_balance']:,.2f}
    
    Analyze and return JSON.
    """
    
    response = llm.invoke([SystemMessage(content=system_message), HumanMessage(content=prompt)])
    
    try:
        content = response.content.strip()
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        fraud_data = json.loads(content)
        fraud_score = fraud_data.get('fraud_score', 30)
        fraud_reason = fraud_data.get('fraud_reason', 'Analysis completed')
    except:
        fraud_score = 30
        fraud_reason = "Error parsing response"
    
    print(f"\n🔍 Fraud Score: {fraud_score}/100")
    print(f"Reason: {fraud_reason}\n")
    
    return {"fraud_score": fraud_score, "fraud_reason": fraud_reason}

def review_suspicious_transaction(state: TransactionState) -> TransactionState:
    """DYNAMIC INTERRUPT: Only pause if fraud score > 60"""
    fraud_score = state['fraud_score']
    
    if fraud_score > 60:
        print(f"\n⚠️  HIGH FRAUD RISK ({fraud_score}) - Pausing for review")
        
        # DYNAMIC INTERRUPT
        analyst_decision = interrupt({
            "message": "Fraud analyst review required",
            "fraud_score": fraud_score,
            "amount": state['transaction_amount']
        })
        
        print(f"\n👤 Analyst Decision: {analyst_decision}\n")
        return {"fraud_analyst_decision": analyst_decision, "final_status": analyst_decision}
    else:
        print(f"\n✅ LOW RISK ({fraud_score}) - Auto-approved\n")
        return {"fraud_analyst_decision": "AUTO_APPROVED", "final_status": "APPROVED"}

def process_transaction(state: TransactionState) -> TransactionState:
    """Process transaction based on decision"""
    status = state['final_status']
    print(f"\n💳 Transaction {status}\n")
    return {"final_status": f"Transaction {state['transaction_id']} {status}"}

print("✅ Nodes defined")

In [ ]:
# Lab 2: Build Workflow

def create_fraud_workflow():
    workflow = StateGraph(TransactionState)
    
    workflow.add_node("analyze_transaction", analyze_transaction)
    workflow.add_node("review_suspicious", review_suspicious_transaction)
    workflow.add_node("process_transaction", process_transaction)
    
    workflow.add_edge(START, "analyze_transaction")
    workflow.add_edge("analyze_transaction", "review_suspicious")
    workflow.add_edge("review_suspicious", "process_transaction")
    workflow.add_edge("process_transaction", END)
    
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

print("✅ Workflow created")

In [ ]:
# Lab 2: Test Suspicious Transaction

app = create_fraud_workflow()
thread_1 = {"configurable": {"thread_id": "txn_001"}}

suspicious_txn = {
    "transaction_id": "TXN-001",
    "account_holder": "John Smith",
    "transaction_amount": 8500.00,
    "merchant": "Electronics - Moscow",
    "location": "Moscow, Russia",
    "transaction_type": "Purchase",
    "account_balance": 12000.00,
    "recent_transactions": [{"amount": 45, "merchant": "Coffee"}],
    "fraud_score": 0.0,
    "fraud_reason": "",
    "fraud_analyst_decision": "",
    "final_status": "",
    "messages": []
}

print("="*80)
print("SCENARIO 1: Suspicious Transaction")
print("="*80)

for event in app.stream(suspicious_txn, thread_1, stream_mode="values"):
    pass

# Check if interrupted
state = app.get_state(thread_1)
if state.tasks:
    print("⏸️  Paused for analyst review")
    print(f"Interrupt data: {state.tasks[0].interrupts[0].value}")

In [ ]:
# Lab 2: Resume with Analyst Decision

print("\n👤 Fraud Analyst approves after verification\n")

# Resume with Command
for event in app.stream(Command(resume="APPROVED"), thread_1, stream_mode="values"):
    pass

print("✅ Transaction processed")

In [ ]:
# Lab 2: Test Normal Transaction (No Interrupt)

thread_2 = {"configurable": {"thread_id": "txn_002"}}

normal_txn = {
    "transaction_id": "TXN-002",
    "account_holder": "Jane Doe",
    "transaction_amount": 85.50,
    "merchant": "Grocery Store",
    "location": "New York, USA",
    "transaction_type": "Purchase",
    "account_balance": 5500.00,
    "recent_transactions": [{"amount": 42, "merchant": "Restaurant"}],
    "fraud_score": 0.0,
    "fraud_reason": "",
    "fraud_analyst_decision": "",
    "final_status": "",
    "messages": []
}

print("\n" + "="*80)
print("SCENARIO 2: Normal Transaction")
print("="*80)

for event in app.stream(normal_txn, thread_2, stream_mode="values"):
    pass

print("✅ Completed without human intervention")

### Lab 2 Exercises:
1. Lower fraud threshold to 50 to see more interrupts
2. Add second interrupt for amounts > $10,000
3. Implement three-tier review system
4. Add rejection reason capture with another interrupt

---
## Complete Training Package

This notebook demonstrates all key human-in-the-loop patterns:

✅ **Static Interrupts** - `interrupt_before` and `interrupt_after`  
✅ **Dynamic Interrupts** - `interrupt()` function with conditional logic  
✅ **State Editing** - `update_state()` for corrections  
✅ **Multi-turn Conversations** - Using `Command(resume=...)` 

### Additional Resources:
- [LangGraph Documentation](https://python.langchain.com/docs/langgraph)
- [Human-in-the-Loop Guide](https://langchain-ai.github.io/langgraph/concepts/human_in_the_loop/)
- [Groq Documentation](https://console.groq.com/docs)